# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os
import joblib
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import Model

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import logging
import time

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RVVUS3GSP to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
quick-starts-ws-143184
aml-quickstarts-143184
southcentralus
9b72f9e6-56c5-4c16-991b-19c652994860


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
# choose a name for experiment
experiment_name = 'capstoneautoml'

experiment=Experiment(ws, experiment_name)

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
amlcompute_cluster_name="runautoml"

try:
    aml_compute=ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config=AmlCompute.provisioning_configuration(vm_size="Standard_D3_V2",max_nodes=4)
    aml_compute=ComputeTarget.create(ws,amlcompute_cluster_name,compute_config)

aml_compute.wait_for_completion(show_output=True)

Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
auto_ds_path="https://raw.githubusercontent.com/kgpyi/Machine-Learning/master/Udacity/Machine%20Learning%20Engineer%20with%20Microsoft%20Azure%20Nanodegree%20Program/Capstone%20Project/heart_failure_clinical_records_dataset.csv"
ds=TabularDatasetFactory.from_delimited_files(path=auto_ds_path)

In [9]:
df = ds.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [10]:
# TODO: Put your automl settings here
automl_settings = {
    "n_cross_validations":8,
    "primary_metric":'accuracy',
    "enable_early_stopping":True,
    "experiment_timeout_minutes":20,
    "max_concurrent_iterations":15,
   }

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task='classification',
    compute_target='runautoml',
    training_data=ds,
    label_column_name='DEATH_EVENT',
    **automl_settings
)

In [11]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on runautoml with default configuration
Running on remote compute: runautoml


Experiment,Id,Type,Status,Details Page,Docs Page
capstoneautoml,AutoML_35c76826-52d0-4164-9bbc-032603e68247,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
capstoneautoml,AutoML_35c76826-52d0-4164-9bbc-032603e68247,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_35c76826-52d0-4164-9bbc-032603e68247',
 'target': 'runautoml',
 'status': 'Completed',
 'startTimeUtc': '2021-04-19T20:05:25.150024Z',
 'endTimeUtc': '2021-04-19T20:30:49.479004Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '8',
  'target': 'runautoml',
  'AMLSettingsJsonString': '{"path":null,"name":"capstoneautoml","subscription_id":"9b72f9e6-56c5-4c16-991b-19c652994860","resource_group":"aml-quickstarts-143184","workspace_name":"quick-starts-ws-143184","region":"southcentralus","compute_target":"runautoml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":0.0,"validation_size":0.0,"n_c

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
best_remote_run,best_automl_model=remote_run.get_output()
best_run_metrics=remote_run.get_metrics()
for primary_metric in best_run_metrics:
    metric=best_run_metrics[primary_metric]
    print(primary_metric,metric)

experiment_status ['DatasetEvaluation', 'FeaturesGeneration', 'DatasetFeaturization', 'DatasetFeaturizationCompleted', 'DatasetCrossValidationSplit', 'ModelSelection', 'BestRunExplainModel', 'ModelExplanationDataSetSetup', 'PickSurrogateModel', 'EngineeredFeatureExplanations', 'EngineeredFeatureExplanations', 'RawFeaturesExplanations', 'RawFeaturesExplanations', 'BestRunExplainModel']
experiment_status_description ['Gathering dataset statistics.', 'Generating features for the dataset.', 'Beginning to fit featurizers and featurize the dataset.', 'Completed fit featurizers and featurizing the dataset.', 'Generating individually featurized CV splits.', 'Beginning model selection.', 'Best run model explanations started', 'Model explanations data setup completed', 'Choosing LightGBM as the surrogate model for explanations', 'Computation of engineered features started', 'Computation of engineered features completed', 'Computation of raw features started', 'Computation of raw features complet

In [14]:
print(best_remote_run)
best_remote_run

Run(Experiment: capstoneautoml,
Id: AutoML_35c76826-52d0-4164-9bbc-032603e68247_58,
Type: azureml.scriptrun,
Status: Completed)


Experiment,Id,Type,Status,Details Page,Docs Page
capstoneautoml,AutoML_35c76826-52d0-4164-9bbc-032603e68247_58,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [15]:
import pickle
best_remote_run,best_automl_model=remote_run.get_output()
pkl_filename = "automl_model.pkl"
with open(pkl_filename,'wb') as f:
    pickle.dump(best_automl_model,f)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
model = Model.register(workspace = ws,
                        model_name = "Heart_Failure_Prediction",
                        model_path = "./automl_model.pkl",
                        description = "AutoML Model which was best according to accuracy",)

Registering model Heart_Failure_Prediction


In [18]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment

#defining an inference configuration
service_name = 'heart-failure-automl'

#setting up a curated environment
from azureml.automl.core.shared import constants
best_remote_run.download_file(constants.CONDA_ENV_FILE_PATH, 'AzureML-AutoML.yml')
env = Environment.get(ws, name="AzureML-AutoML")

#combining scoring script and environment in inference configuration
inference_config = InferenceConfig(entry_script='scoring.py', environment=env)

#deploy the model as a webservice
from azureml.core.webservice import LocalWebservice, Webservice

#set deployment configuration
local_deployment_config = LocalWebservice.deploy_configuration(port=8000)
service = Model.deploy(ws, "heart-failure-automl", [model], inference_config, local_deployment_config)

print(service.state)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, enable_app_insights=True, auth_enabled=True) #service.update

service = Model.deploy(workspace=ws,
                        name='heart-failure-automl',
                        models=[model],
                        inference_config=inference_config,
                        deployment_config=aci_config,
                        overwrite=True)
service.wait_for_deployment(show_output=True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_39c362815a3a9e28eebe0c654b193f75
 ---> 5640627ca31e
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 6c9ec8d73849
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjliNzJmOWU2LTU2YzUtNGMxNi05OTFiLTE5YzY1Mjk5NDg2MCIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTE0MzE4NCIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTE0MzE4NCIsIndvcmtzcGFjZUlkIjoiZDZkYmZlNDktZDNhYi00OTFkLTlkNWUtYmExYmE3MzVlZDQyIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 9fd42549e727
 ---> 6603a2a9672f
Step 4/5 : RUN mv '/var/azureml-app/tmpjwxngc9q.py' /var/azureml-app/main.py
 ---> Running in 968f7bab4380
 ---> e175b5281ece
Step 5/5 : 

In [19]:
print(service.swagger_uri)
print(service.scoring_uri)

http://ff8e5c52-a5e6-43f0-a5ab-7476584d260c.southcentralus.azurecontainer.io/swagger.json
http://ff8e5c52-a5e6-43f0-a5ab-7476584d260c.southcentralus.azurecontainer.io/score


In [20]:
primary, secondary= service.get_keys()
print(primary)

aZHHIwUsczmsSQlbIx67mtgW9G9m1Vfc


TODO: In the cell below, send a request to the web service you deployed to test it.

In [22]:
import requests
import json

# URL for the web service
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
key = primary

# Two sets of data to score, so we get two results back
data = {"data":
        [
            {
               "age":45,
               "anaemia":1,
               "creatinine_phosphokinase":7861,
               "diabetes":1,
               "ejection_fraction":49,
               "high_blood_pressure":0,
               "platelets":276000,
               "serum_creatinine":1,
               "serum_sodium":140,
               "sex":0,
               "smoking":1,
               "time":16
               },
            {
               "age":51,
               "anaemia":0,
               "creatinine_phosphokinase":78,
               "diabetes":0,
               "ejection_fraction":50,
               "high_blood_pressure":0,
               "platelets":406000,
               "serum_creatinine":0.7,
               "serum_sodium":140,
               "sex":1,
               "smoking":0,
               "time":79
                }
        ]
        }

# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {primary}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

"{\"result\": [1, 0]}"


TODO: In the cell below, print the logs of the web service and delete the service

In [23]:

#logs of the webservice
logs=service.get_logs()
print(logs)

2021-04-19T20:52:56,037359800+00:00 - iot-server/run 
2021-04-19T20:52:56,055548600+00:00 - gunicorn/run 
2021-04-19T20:52:56,057517700+00:00 - rsyslog/run 
2021-04-19T20:52:56,075367900+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [24]:

#delete the deployed webservice
service.delete()